# METADATA CLIENT SCRIPT OPTIMIZE

In [75]:
from ppadb.client import Client as AdbClient
import pandas as pd
import os
from confluent_kafka import Producer
from confluent_kafka import Consumer, KafkaException, TopicPartition, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic
#from confluent_kafka.admin import AdminException
import subprocess
import time
import platform
from exif import Image
import glob
from time import sleep
import shutil
import socket
import sys

In [76]:
###################### Function ######################
# 1. Function push message to kafka topic
def push_message_to_kafka(message,ip,port,topic):
    bootstrap_servers = ip+":"+port

    # Kafka producer configuration
    producer_config = {
        'bootstrap.servers': bootstrap_servers
    }

    # Create a Kafka producer
    producer = Producer(producer_config)

    try:
        # Produce the message
        producer.produce(topic=topic, value=message)
        producer.flush()  # Wait for the message to be sent
        print("Message sent successfully.")
    except Exception as e:
        print(f"Failed to send message to Kafka: {e}")
# 2. Function redirect taffic go to MITM Proxy'
def set_http_proxy(server_ip,port):
    try:
        command = f"adb shell settings put global http_proxy {server_ip}:{port}"
        subprocess.run(command, shell=True, check=True)
        print(f"HTTP proxy set to {server_ip}:{port} successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to set HTTP proxy: {e}")
# 3. Function lauch app
def run_app_by_package_name(package_name):
    try:
        command = f"adb shell monkey -p {package_name} 1"
        subprocess.run(command, shell=True, check=True)
        print(f"The app with package name '{package_name}' has been launched.")
        return 1
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to run the app with package name '{package_name}': {e}")
        return 0
# 4. Function check app running by package name
def check_app_running(package_name):
    try:
        adb_command = "adb" if platform.system() == "Windows" else "./adb"
        command = f'{adb_command} shell "pidof {package_name}"'
        output = subprocess.check_output(command, shell=True).decode().strip()

        if output:
            print(f"The app with package name '{package_name}' is running.")
            return True
        else:
            print(f"The app with package name '{package_name}' is not running.")
            return False
    except subprocess.CalledProcessError as e:
        if e.returncode == 1:
            print(f"The app with package name '{package_name}' is not running.")
            return False
        else:
            print(f"Failed to execute the 'adb' command. Error: {e.output.decode().strip()}")
            return False
# 15. Function check number of file
def count_files_in_directory(directory_path):
    if os.path.isdir(directory_path):
        file_count = len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])
        return file_count
    else:
        return 0
# 5. Function receive MITM Proxy capture
def consume_and_write_to_files(topic, directory, kafka_ip, kafka_port):
    consumer = Consumer({
        'bootstrap.servers': f'{kafka_ip}:{kafka_port}',
        'group.id': 'my-consumer-group',
        'auto.offset.reset': 'earliest',
        'enable.auto.commit': False
    })
    consumer.subscribe([topic])  # Replace with the desired Kafka topic

    try:
        while True:
            message = consumer.poll(timeout=1.0)

            if message is None:
                if count_files_in_directory(directory)==0:
                    continue
                print("Kafka status :there is no more message")
                if consumer.committed(consumer.assignment()) == consumer.position(consumer.assignment()):
                    # All messages received, stop subscribing
                    print("Kafka status :receive all message")
                    consumer.unsubscribe()
                    break

            if message.error():
                continue
#                 if message.error().code() == KafkaException._PARTITION_EOF:
#                     continue
#                 else:
#                     print(f"Consumer error: {message.error()}")
#                     continue

            value = message.value()
            # Write the message value to a file in the specified directory
            file_path = os.path.join(directory, f"message_{message.offset()}")
            with open(file_path, 'wb') as file:
                print("---Write file:"+str(file_path))
                file.write(value)
                time.sleep(2)
            consumer.commit(message)  # Commit the consumed message

    finally:
        consumer.unsubscribe()
        consumer.close()
# 6. Function create directory if not exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")
# 7. Funtion Remove all bytes file
def delete_files_in_directory(directory):
    file_list = os.listdir(directory)
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
# 8. Function change file extension
def change_extension(directory, new_extension):
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            base_name, _ = os.path.splitext(filename)
            new_filename = base_name + new_extension
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
# 9. Function all image matadata in directory
def getImageMetaData(image_path:str):
    file_paths = glob.glob(image_path + '\\*')
    img = ""
    metadata_array = []
    for file in file_paths:
        with open(file, 'rb') as image_file:
            img = Image(image_file) 
        haveMetaData = img.has_exif
        if(haveMetaData):
            meta_key = img.list_all()
            for i in range(len(meta_key)):
                try:
                    metadata = str(meta_key[i]) +":"+ str(img.get(meta_key[i]))
                    metadata_array.append(metadata)
                except Exception as e:
                    print(f"Error occurred: {str(e)}")
    return metadata_array
# 10. Checking metadata of each image
def checkLeakingMetaData(downloadImageMetaDataList: list, checkingMetaDataList: list):
    leak_info = []
    for a in range(len(checkingMetaDataList)):
        checkingMetaData = checkingMetaDataList[a]
        for b in range(len(downloadImageMetaDataList)):
            downloadImageMetaData = downloadImageMetaDataList[b]
            if(checkingMetaData in downloadImageMetaData):
                if a==0 or a==4 or a==8:
                    leak_info.append("Leak datetime taken")
                    break
                elif a==1 or a==5 or a==9:
                    leak_info.append("Leak gps_latitude")                 
                    break
                elif a==2 or a==6 or a==10:
                    leak_info.append("Leak gps_longitude")
                    break
                elif a==3 or a==7 or a==11:
                    leak_info.append("Leak gps_altitude")
                    break
                elif a==12:
                    leak_info.append("Leak smart phone model - SM-M515F")
                    break
                elif a==13:
                    leak_info.append("Leak smart phone brand - samsung")
                    break
                elif a==14:
                    leak_info.append("Leak smart phone software - M515FXXS4DWA3")
                    break
                else:
                    leak_info.append("No leak information")
                    break
    return leak_info
# 11. Function convert list to string
def listToString(listData: list):
    listToStr = '\n'.join([str(elem) for elem in listData])
    return listToStr
# 12. Function checking running service
def check_port_running(ip, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(3)  # Set a timeout value for the connection
        result = sock.connect_ex((ip, port))
        if result == 0:
            # Port is open and running
            return True
        else:
            # Port is not open
            return False
    except socket.error as e:
        print(f"Error occurred while checking port: {e}")
        return False
# 13. Function delete kafka topic
def delete_kafka_topic(kafka_ip, kafka_port, topic_name):
    # Create the AdminClient instance
    bootstrap_servers = f"{kafka_ip}:{kafka_port}"
    admin_client = AdminClient({"bootstrap.servers": bootstrap_servers})

    # Delete the topic
    deletion_result = admin_client.delete_topics([topic_name])
    time.sleep(2)
    # Wait for the topic deletion to finish
    deletion_result[topic_name].result()

    print(f"Topic '{topic_name}' deleted successfully")
# 14. Function create kafka topic
def check_create_topic(kafka_ip, kafka_port,topic_name):
    #bootstrap_servers = 'localhost:9092'  # Replace with your Kafka bootstrap servers
    #admin_client = AdminClient({'bootstrap.servers': bootstrap_servers})
    # Create the AdminClient configuration
    admin_conf = {
        'bootstrap.servers': f"{kafka_ip}:{kafka_port}"
    }
    # Create the AdminClient instance
    admin_client = AdminClient(admin_conf)
    topic_exists = False

    try:
        # Create the topic
        topic_config = {'num_partitions': 1, 'replication_factor': 1}
        topic = NewTopic(topic_name, **topic_config)
        admin_client.create_topics([topic])
        print(f"Topic '{topic_name}' created.")
        
        # Introduce a small delay for topic metadata propagation
        time.sleep(1)

        # Check if the topic exists again
        topic_metadata = admin_client.list_topics(timeout=5).topics
        if topic_name in topic_metadata:
            topic_exists = True
            print(f"Topic '{topic_name}' exists.")
    except KafkaException as e:
        print(f"Error creating or checking topic '{topic_name}': {e}")
# 15. Function un-install APK
def uninstall_apk(device, package_name):
    try:
        # Build the uninstall command
        uninstall_command = f"pm uninstall {package_name}"
        # Execute the uninstall command on the device
        result = device.shell(uninstall_command)
        if "Success" in result:
            print(f"APK {package_name} was successfully uninstalled.")
        else:
            print(f"Failed to uninstall APK {package_name}. Error: {result}")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

In [77]:
# 0. Server variable
kafka_ip = "27.71.27.174" #"192.168.1.4" #"27.71.27.174"#"54.169.58.0"#"192.168.1.25"
kafka_port = "9092"
tester_name = "lam"
kafka_topic = "metadata-"+tester_name
receive_topic = "bytes-"+tester_name
mitm_ip = "27.71.27.174" #"192.168.1.4" #"27.71.27.174"#"54.169.58.0"
#mitm_ip = "54.169.58.0"#"193.206.183.20"
mitm_port = 8080 

In [78]:
# 1. Connect to device
# 1.1 Create directory if not exist and clear all data
receive_directory = "./receive_bytes"
create_directory(receive_directory)
delete_files_in_directory(receive_directory)
# 1.2
client = ""
device = ""
#device_name = "emulator-5554"
device_name = "RF8NB1NDPXA" #Galaxy M51
#device_name = "R58W70GXT4A" #Galaxy A04e
device_ip = "127.0.0.1"
device_port = 5037
try:
    client = AdbClient(host=device_ip, port=device_port)
    device = client.device(device_name)
except:
    print("Device error")
if(device!=None):
    print("Device connected")
else:
    print("Device disconnected")
# 1.3 Create kafka topic
check_create_topic(kafka_ip, kafka_port,kafka_topic)

Directory './receive_bytes' already exists.
Device connected
Topic 'metadata-lam' created.
Topic 'metadata-lam' exists.


In [79]:
# 2. Read APK information
# 2.1 Define 
df = ""
category_name = "photography" #  personalization
apk_csv_path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"+category_name+"_apk_popular.csv"
#apk_csv_path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\[9-8-2023]photography_apk_popular_3501_to_4500.csv"
apk_file_path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"
# print(apk_csv_path)
# print(apk_file_path)
# 2.2 Read apk information from CSV file
file_exists = os.path.exists(apk_csv_path)
if(file_exists):
    df = pd.read_csv(apk_csv_path)
apk_file_name_arr = df["apkfile_name"]
apk_pkg_name_arr = df["pkg_name"]
# 2.3 Get downloaded APK file
downloaded_apk = []
downloaded_package = []
print("Length: "+str(len(apk_file_name_arr)))
for apk_num in range(len(apk_file_name_arr)):
    apk_file_name = str(apk_file_name_arr[apk_num])
    apk_pkg_name = str(apk_pkg_name_arr[apk_num])
    downloaded_apk.append(apk_file_name)
    downloaded_package.append(apk_pkg_name)

Length: 5548


In [80]:
###### 3. Install APK file
# 3.1 Redirect traffic go through MITM Proxy
set_http_proxy(mitm_ip,mitm_port)
# 3.2 Install next APK and start MITM Proxy server
i = 5547 #1883 #702  # # Start from 0 to Length #119 error
print("Current loop variable i = "+str(i))
if(device.is_installed(downloaded_package[i])!=True):
    try:
        device.install(apk_file_path+downloaded_apk[i])
        print("Apk "+ downloaded_apk[i] + " installed success")
        push_message_to_kafka("start",kafka_ip,kafka_port,kafka_topic)
    except Exception as e:
        df.loc[df['pkg_name'] == downloaded_package[i], 'metadata_leak'] = "Install fail"
        df.to_csv(apk_csv_path,index=False)
        print(f"Error occurred: {str(e)}")
else:
    push_message_to_kafka("start",kafka_ip,kafka_port,kafka_topic)
    print("Apk "+ downloaded_apk[i] + " is installed")
# 3.3 Check MITM Proxy service is running
time.sleep(10)
mitmProxyStatus = check_port_running(mitm_ip,mitm_port)
print(mitmProxyStatus)
if(not mitmProxyStatus):
    print("MITM Proxy is not running.")
    sys.exit("Alert: Stopping Jupyter Notebook.")
else:
    print("MITM Proxy is running.")

HTTP proxy set to 27.71.27.174:8080 successfully.
Current loop variable i = 5547
Message sent successfully.
Apk nan is installed
True
MITM Proxy is running.


In [81]:
# 4. Lauch App
package_name = downloaded_package[i]
runStatus = run_app_by_package_name(package_name)
print(runStatus)
if(runStatus !=1):
        df.loc[df['pkg_name'] == package_name, 'metadata_leak'] = "Install fail"
        df.to_csv(apk_csv_path,index=False)
        sys.exit("APK install fail! Please go to next APK!")

The app with package name 'com.amazon.clouddrive.photos' has been launched.
1


In [82]:
# 5. Checking APP status
while True:
        if check_app_running(package_name):
            print("APK is running. Continuing to wait...")
        else:
            print("APK is stopped. Stop MITM Proxy service")
            push_message_to_kafka("stop",kafka_ip,kafka_port,kafka_topic)
            time.sleep(5)
            set_http_proxy("",0)
            consume_and_write_to_files(receive_topic, "receive_bytes",kafka_ip,kafka_port)
            time.sleep(10)
            break
        time.sleep(1)  # Wait for 1 second before checking again

The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon

The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon

The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon

The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon

The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon.clouddrive.photos' is running.
APK is running. Continuing to wait...
The app with package name 'com.amazon

In [83]:
# 6. Change file extension
change_extension(receive_directory,".jpg")
# 7. Stop redirect traffic to MITM Proxy
#set_http_proxy("",0)

In [84]:
try:
    # 8. Check leak sensitive metadata
    image_path = "C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\receive_bytes\\"
    # This is metadta of image LANDSCAPE_ORIGIN.jpg
    checkingMetaDataList = ["2023:03:08 17:05:59",
                        "(45.0, 47.0, 41.56404)",
                        "(8.0, 50.0, 41.838)",
                        "444.0",
                        "2023:05:01 22:39:37",
                        "(45.0, 47.0, 38.922)",
                        "(8.0, 50.0, 48.507719)",
                        "439.0",
                        "2023:03:08 11:28:25",
                        "(45.0, 47.0, 53.199599)",
                        "(8.0, 50.0, 53.71764)",
                        "447.0",
                        "SM-M515F",
                        "samsung",
                        "M515FXXS4DWA3"
                       ]
    if(len(image_path)>0):
        # 8.1 Get all metadata in image
        downloadImageMetaDataList = getImageMetaData(image_path)
        # 8.2.Check leak sensitive metadata or not
        leak_info = checkLeakingMetaData(downloadImageMetaDataList,checkingMetaDataList)
        print(leak_info)
        # 8.3 Write info to file
        if(len(leak_info)>0):
            df.loc[df['pkg_name'] == package_name, 'metadata_leak'] = listToString(leak_info)
            df.to_csv(apk_csv_path,index=False)
        elif(len(leak_info)==0):
            df.loc[df['pkg_name'] == package_name, 'metadata_leak'] = "No Leak Sensitive Metadata"
            df.to_csv(apk_csv_path,index=False)
    else:
        print("Did not receive any captured package from Server Service. Please check!")
except Exception as e:
    print(f"Error occurred: {str(e)}")
    df.loc[df['pkg_name'] == package_name, 'metadata_leak'] = "Install fail"
    df.to_csv(apk_csv_path,index=False)
finally:
    # 9. Delete Kafka topic
    time.sleep(5)
    delete_kafka_topic(kafka_ip, kafka_port, receive_topic)
    #delete_kafka_topic(kafka_ip, kafka_port, "metadata")

Error occurred: 0 is not a valid Orientation
Error occurred: 0 is not a valid Orientation
Error occurred: 

+--------+------------+-------+-------+------------------------+
| Offset | Access     | Value | Bytes | Format                 |
+--------+------------+-------+-------+------------------------+
|        |            |       |       | TiffHeader (Structure) |
| 0      | byte_order | 5636  | 16 04 | tiff_byte_order        |
+--------+------------+-------+-------+------------------------+

ValueError occurred during unpack operation:

5636 is not a valid TiffByteOrder
Topic 'bytes-lam' deleted successfully


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ASUS\\anaconda3\\phd_implement\\photography_category\\photography_apk_popular.csv'

In [ ]:
#9. Un-install APK
#uninstall_apk(device,package_name)